In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload


import sys, os, inspect
import sys
sys.path.append('/Users/emigardiner/VICO/pjams-ionization/pjams/')

from zeusmp_snapshot_reader import read_zeusmp_snapshot
from zeusmp_snapshot_reader import ScaleFactors
from snapshot import snapshot 
from basic_snapshot import basic_snapshot, VICO_loc
import plot as plot
import ionfrac 
from ionfrac import VMIN, VMAX
# import flux

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm
from matplotlib.colors import TwoSlopeNorm

from PIL import Image, ImageDraw, ImageFont
from IPython.display import Image as DisplayImage

In [ ]:
INCLUDE_4000 = True

In [ ]:
# VICO_loc = '/Users/emigardiner/VICO/pjams-ionization'
# MYFONT = ImageFont.truetype('/System/Library/Fonts/Supplemental/Times New Roman.ttf', 65)
# SMALLFONT = ImageFont.truetype('/System/Library/Fonts/Supplemental/Times New Roman.ttf', 55)
    

# # INPUTS
# FREQS = np.array([.01, .05, .1, .5, 1, 5.3, 23, 43, 100, 230 ]) # GHz
# FREQS *= 10**9 # Hz

# r_kpc = 1
# heights_and_scales = np.load(VICO_loc+'/Data/heights_and_scales.npz')
# scales = heights_and_scales['scales'] # AU
# heights = heights_and_scales['heights'] # AU  

# colors = plot.COLORS
# colors_cont = ['darkorange', 'red', 'magenta']

# Read in snaps

In [ ]:
nums = np.array([4, 9, 21, 39, 54, 68, 94])
years = np.array(['4,000 yrs', '9,000 yrs', '21,000 yrs', '39,000 yrs', '54,000 yrs', '68,000 yrs', '94,000 yrs'])
masss = np.array([r'1.4 M$_\odot$', r'2 M$_\odot$', r'4 M$_\odot$',
                   r'8 M$_\odot$', r'12 M$_\odot$', r'16 M$_\odot$',
                   r'24 M$_\odot$'])

r_kpc = 1
heights_and_scales = np.load(VICO_loc+'/Data/heights_and_scales.npz')
scales = heights_and_scales['scales'] # AU
heights = heights_and_scales['heights'] # AU  

In [ ]:
snaps = np.empty_like(nums, dtype=snapshot)
for ii, num in enumerate(nums):
    snaps[ii] = snapshot(snap=num, name = ('Snap%03d_n' % num), read_zeusmp = False)
    # shot[ii].load_shock_variables()
    snaps[ii].load_fluxes()

# Load Ionization Fractions

In [ ]:
# def load_average_ionfrac_array(Snap, debug=False):
#     data_path = (VICO_loc+'/Data/'+Snap.name+'/')
#     loaded_average_ionfracs = np.load((data_path+Snap.name+'_average_ionfrac_arrays.npz'))
#     Snap.ionfrac_mass = loaded_average_ionfracs['ionfrac_mass']
#     Snap.ionfrac_vol =  loaded_average_ionfracs['ionfrac_vol']
#     Snap.ionfrac_emis = loaded_average_ionfracs['ionfrac_emis']
#     if debug: print(Snap.name + ' ionfrac arrays loaded')
    
# def load_average_ionfrac_array_ratio(Snap, const=False, debug=False):
#     data_path = (VICO_loc+'/Data/'+Snap.name+'/')
#     loaded_average_ionfracs = np.load((data_path+Snap.name+'_average_ionfrac_arrays_const'+str(const)+'.npz'))
#     Snap.ionfrac_mass_ratio = loaded_average_ionfracs['ionfrac_mass']
#     Snap.ionfrac_vol_ratio =  loaded_average_ionfracs['ionfrac_vol']
#     Snap.ionfrac_emis_ratio = loaded_average_ionfracs['ionfrac_emis']
#     if debug: print(Snap.name + ' ionfrac_ratio arrays loaded')

In [ ]:
for ss in range(0,len(snaps)):
    ionfrac.load_average_ionfrac_array(snaps[ss])
    ionfrac.load_average_ionfrac_array_ratio(snaps[ss], const=False)
    
# Maximum Velocities for Contours
for s in range(len(snaps)): 
    for v in ionfrac.v_mins:
        ionfrac.max_velocity_proj_array(snaps[s], v)

# Plot

## plotting functions

In [ ]:
# # Ionization Fraction Methods
# import matplotlib.colors as colors
# saveloc = '/Users/emigardiner/VICO/pjams-ionization/pillowfiles/ionfrac_fig'

# def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
#     new_cmap = colors.LinearSegmentedColormap.from_list(
#         'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
#         cmap(np.linspace(minval, maxval, n)))
#     return new_cmap

# cmap = plt.get_cmap('jet')
# new_cmap = ionfrac.truncate_colormap(cmap, 0.1, 1)

## Setup Colorbar Files

In [ ]:
vert_cname = ionfrac.vol_ionfrac_pcolormesh(snaps[1], years[1], scale=scales[1], vmin=VMIN, vmax=VMAX, v=0,
                            show_xlabels=False, show_ylabels=False, 
                            show_cbar=True, vertical_cbar=True,
                            saveloc=ionfrac.SAVELOC+'/cbars')
vert_crop = 20
horiz_cname = ionfrac.vol_ionfrac_pcolormesh(snaps[1], years[1], scale=scales[1], vmin=VMIN, vmax=VMAX, v=0,
                            show_xlabels=False, show_ylabels=False, 
                            show_cbar=True, vertical_cbar=False,
                            saveloc=ionfrac.SAVELOC+'/cbars')
horiz_crop = 30

# PIL Images

#### 1A PIL Figure: 25000au no ratio

In [ ]:
nrows = len(snaps) if INCLUDE_4000 else len(snaps) - 1
ncols = 9
scale = 25000
imgfiles = np.empty((nrows, ncols), dtype=object)
v_mins = ionfrac.v_mins
r=0
for s in range(len(snaps) - nrows,len(snaps)):
    c=0
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        if(v==0): 
            show_ylabels = True
            name = masss[s]
        else: 
            show_ylabels = False
            name = None
        filename = ionfrac.mass_ionfrac_pcolormesh(snaps[s], years[s], scale, vmin=VMIN, vmax=VMAX, v=v,
                               show_xlabels=show_xlabels, show_ylabels=show_ylabels,
                               name=name, xx_name=0.02, yy_name=0.05, namefs=30)
        imgfiles[r,c] = filename
        c+=1
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        show_ylabels = False
        filename = ionfrac.vol_ionfrac_pcolormesh(snaps[s], years[s], scale, vmin=VMIN, vmax=VMAX, v=v,
                              show_xlabels=show_xlabels, show_ylabels=show_ylabels)
        imgfiles[r,c] = filename
        c+=1
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        show_ylabels = False
        filename = ionfrac.emis_ionfrac_pcolormesh(snaps[s], years[s], scale, f=5, vmin=VMIN, vmax=VMAX, v=v,
                               show_xlabels=show_xlabels, show_ylabels=show_ylabels)
        imgfiles[r,c] = filename
        c+=1
    r+=1

imgfiles_arr_25000au_noratio = imgfiles

In [ ]:
allimage = plot.ionfrac_pil_image(imgfiles_arr_25000au_noratio, horiz_cname, extracrop=horiz_crop, debug=False)
display(allimage)
name = VICO_loc+'/figures/ionfrac_maps/ionfrac_caseA_25000au.png' if INCLUDE_4000 else VICO_loc+'/figures/ionfrac_maps/ionfrac_flooded_25000au.png'
allimage.save(name)

#### 1B PIL Figure: 25000au no ratio, with contours

In [ ]:
nrows = len(snaps) if INCLUDE_4000 else len(snaps) - 1
ncols = 9
scale = 25000
show_contours=True
lw_cont = 3
colors_cont = ['#8080ff', '#ff99ff', 'white']


imgfiles = np.empty((nrows, ncols), dtype=object)
r=0
for s in range(len(snaps)-nrows,len(snaps)):
    c=0
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        if(v==0): 
            show_ylabels = True
            name = masss[s]
        else: 
            show_ylabels = False
            name = None
        filename = ionfrac.mass_ionfrac_pcolormesh(snaps[s], years[s], scale, vmin=VMIN, vmax=VMAX, v=v,
                               show_xlabels=show_xlabels, show_ylabels=show_ylabels,
                               name=name, xx_name=0.02, yy_name=0.05, namefs=30, 
                               show_contours=show_contours, lw_cont=lw_cont, colors_cont=colors_cont)
        imgfiles[r,c] = filename
        c+=1
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        show_ylabels = False
        filename = ionfrac.vol_ionfrac_pcolormesh(snaps[s], years[s], scale, vmin=VMIN, vmax=VMAX, v=v,
                              show_xlabels=show_xlabels, show_ylabels=show_ylabels, 
                               show_contours=show_contours, lw_cont=lw_cont, colors_cont=colors_cont)
        imgfiles[r,c] = filename
        c+=1
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        show_ylabels = False
        filename = ionfrac.emis_ionfrac_pcolormesh(snaps[s], years[s], scale, f=5, vmin=VMIN, vmax=VMAX, v=v,
                               show_xlabels=show_xlabels, show_ylabels=show_ylabels, 
                               show_contours=show_contours, lw_cont=lw_cont, colors_cont=colors_cont)
        imgfiles[r,c] = filename
        c+=1
    r+=1

imgfiles_arr_25000au_noratio_contours = imgfiles

In [ ]:
allimage = plot.ionfrac_pil_image(imgfiles_arr_25000au_noratio_contours, horiz_cname, extracrop=horiz_crop, debug=False)
display(allimage)
name = VICO_loc+'/figures/ionfrac_maps/ionfrac_caseA_25000au_contours.png' if INCLUDE_4000 else VICO_loc+'/figures/ionfrac_maps/ionfrac_flooded_25000au_contours.png'
allimage.save(name)

#### 2 PIL Figure: 4000au no ratio

In [ ]:
nrows = len(snaps) if INCLUDE_4000 else len(snaps) - 1
ncols = 9
scale = 4000
imgfiles = np.empty((nrows, ncols), dtype=object)
r=0
for s in range(len(snaps)-nrows,len(snaps)):
    c=0
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        if(v==0): 
            show_ylabels = True
            name = masss[s]
        else: 
            show_ylabels = False
            name = None
        filename = ionfrac.mass_ionfrac_pcolormesh(snaps[s], years[s], scale, vmin=VMIN, vmax=VMAX, v=v,
                               show_xlabels=show_xlabels, show_ylabels=show_ylabels,
                               name=name, xx_name=0.02, yy_name=0.05, namefs=30)
        imgfiles[r,c] = filename
        c+=1
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        show_ylabels = False
        filename = ionfrac.vol_ionfrac_pcolormesh(snaps[s], years[s], scale, vmin=VMIN, vmax=VMAX, v=v,
                              show_xlabels=show_xlabels, show_ylabels=show_ylabels)
        imgfiles[r,c] = filename
        c+=1
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        show_ylabels = False
        filename = ionfrac.emis_ionfrac_pcolormesh(snaps[s], years[s], scale, f=5, vmin=VMIN, vmax=VMAX, v=v,
                               show_xlabels=show_xlabels, show_ylabels=show_ylabels)
        imgfiles[r,c] = filename
        c+=1
    r+=1

imgfiles_arr_4000au_noratio = imgfiles

In [ ]:
allimage = plot.ionfrac_pil_image(imgfiles_arr_4000au_noratio, horiz_cname, extracrop=horiz_crop, debug=False)
display(allimage)
name = VICO_loc+'/figures/ionfrac_maps/ionfrac_caseA_4000au.png' if INCLUDE_4000 else VICO_loc+'/figures/ionfrac_maps/ionfrac_flooded_4000au.png'
allimage.save(name)

#### 2B PIL Figure: 4000 au no rato with contours

In [ ]:
nrows = len(snaps) if INCLUDE_4000 else len(snaps) - 1
ncols = 9
scale = 4000
show_contours=True
lw_cont = 3
colors_cont = ['#8080ff', '#ff99ff', 'white']

imgfiles = np.empty((nrows, ncols), dtype=object)
r=0
for s in range(len(snaps)-nrows,len(snaps)):
    c=0
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        if(v==0): 
            show_ylabels = True
            name = masss[s]
        else: 
            show_ylabels = False
            name = None
        filename = ionfrac.mass_ionfrac_pcolormesh(snaps[s], years[s], scale, vmin=VMIN, vmax=VMAX, v=v,
                               show_xlabels=show_xlabels, show_ylabels=show_ylabels,
                               name=name, xx_name=0.02, yy_name=0.05, namefs=30, 
                               show_contours=show_contours, lw_cont=lw_cont, colors_cont=colors_cont, )
        imgfiles[r,c] = filename
        c+=1
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        show_ylabels = False
        filename = ionfrac.vol_ionfrac_pcolormesh(snaps[s], years[s], scale, vmin=VMIN, vmax=VMAX, v=v,
                              show_xlabels=show_xlabels, show_ylabels=show_ylabels, 
                               show_contours=show_contours, lw_cont=lw_cont, colors_cont=colors_cont, )
        imgfiles[r,c] = filename
        c+=1
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        show_ylabels = False
        filename = ionfrac.emis_ionfrac_pcolormesh(snaps[s], years[s], scale, f=5, vmin=VMIN, vmax=VMAX, v=v,
                               show_xlabels=show_xlabels, show_ylabels=show_ylabels, 
                               show_contours=show_contours, lw_cont=lw_cont, colors_cont=colors_cont, )
        imgfiles[r,c] = filename
        c+=1
    r+=1

imgfiles_arr_4000au_noratio_contours = imgfiles

In [ ]:
allimage = plot.ionfrac_pil_image(imgfiles_arr_4000au_noratio, horiz_cname, extracrop=horiz_crop, debug=False)
display(allimage)
name = VICO_loc+'/figures/ionfrac_maps/ionfrac_caseA_4000au_contours.png' if INCLUDE_4000 else VICO_loc+'/figures/ionfrac_maps/ionfrac_flooded_4000au_contours.png'
allimage.save(name)

#### 3A: 25000 au Cooling

In [ ]:
#### 1A PIL Figure: 25000au no ratio
nrows = len(snaps) if INCLUDE_4000 else len(snaps) - 1
ncols = 9
scale = 25000
imgfiles = np.empty((nrows, ncols), dtype=object)
r=0
for s in range(len(snaps)-nrows,len(snaps)):
    c=0
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        if(v==0): 
            show_ylabels = True
            name = masss[s]
        else: 
            show_ylabels = False
            name = None
        filename =ionfrac. mass_ionfrac_const_pcolormesh(snaps[s], years[s], scale, vmin=VMIN, vmax=VMAX, v=v,
                               show_xlabels=show_xlabels, show_ylabels=show_ylabels,
                               name=name, xx_name=0.02, yy_name=0.05, namefs=30)
        imgfiles[r,c] = filename
        c+=1
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        show_ylabels = False
        filename = ionfrac.vol_ionfrac_const_pcolormesh(snaps[s], years[s], scale, vmin=VMIN, vmax=VMAX, v=v,
                              show_xlabels=show_xlabels, show_ylabels=show_ylabels)
        imgfiles[r,c] = filename
        c+=1
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        show_ylabels = False
        filename = ionfrac.emis_ionfrac_const_pcolormesh(snaps[s], years[s], scale, f=5, vmin=VMIN, vmax=VMAX, v=v,
                               show_xlabels=show_xlabels, show_ylabels=show_ylabels)
        imgfiles[r,c] = filename
        c+=1
    r+=1

imgfiles_arr_25000au_ratio = imgfiles

In [ ]:
allimage = plot.ionfrac_pil_image(imgfiles_arr_25000au_ratio, horiz_cname, extracrop=horiz_crop, debug=False)
display(allimage)
name = VICO_loc+'/figures/ionfrac_maps/ionfrac_caseB_25000au.png' if INCLUDE_4000 else VICO_loc+'/figures/ionfrac_maps/ionfrac_cooling_25000au.png'
allimage.save(name)

#### 3B: 25000 au Cooling with contours

In [ ]:
nrows = len(snaps) if INCLUDE_4000 else len(snaps) - 1
ncols = 9
scale = 25000
show_contours=True
lw_cont = 3
colors_cont = ['#8080ff', '#ff99ff', 'white']


imgfiles = np.empty((nrows, ncols), dtype=object)
r=0
for s in range(len(snaps)-nrows,len(snaps)):
    c=0
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        if(v==0): 
            show_ylabels = True
            name = masss[s]
        else: 
            show_ylabels = False
            name = None
        filename = ionfrac.mass_ionfrac_pcolormesh(snaps[s], years[s], scale, vmin=VMIN, vmax=VMAX, v=v,
                               show_xlabels=show_xlabels, show_ylabels=show_ylabels,
                               name=name, xx_name=0.02, yy_name=0.05, namefs=30, 
                               show_contours=show_contours, lw_cont=lw_cont, colors_cont=colors_cont)
        imgfiles[r,c] = filename
        c+=1
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        show_ylabels = False
        filename = ionfrac.vol_ionfrac_pcolormesh(snaps[s], years[s], scale, vmin=VMIN, vmax=VMAX, v=v,
                              show_xlabels=show_xlabels, show_ylabels=show_ylabels, 
                               show_contours=show_contours, lw_cont=lw_cont, colors_cont=colors_cont)
        imgfiles[r,c] = filename
        c+=1
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        show_ylabels = False
        filename = ionfrac.emis_ionfrac_pcolormesh(snaps[s], years[s], scale, f=5, vmin=VMIN, vmax=VMAX, v=v,
                               show_xlabels=show_xlabels, show_ylabels=show_ylabels, 
                               show_contours=show_contours, lw_cont=lw_cont, colors_cont=colors_cont)
        imgfiles[r,c] = filename
        c+=1
    r+=1

imgfiles_arr_25000au_ratio_contours = imgfiles

In [ ]:
allimage = plot.ionfrac_pil_image(imgfiles_arr_25000au_ratio_contours,horiz_cname, extracrop=horiz_crop, debug=False)
display(allimage)
name = VICO_loc+'/figures/ionfrac_maps/ionfrac_caseB_25000au_contours.png' if INCLUDE_4000 else VICO_loc+'/figures/ionfrac_maps/ionfrac_cooling_25000au_contours.png'
allimage.save(name)

#### 4A: 4000 au Cooling

In [ ]:
#### 1A PIL Figure: 25000au no ratio
nrows = len(snaps) if INCLUDE_4000 else len(snaps) - 1
ncols = 9
scale = 4000
imgfiles = np.empty((nrows, ncols), dtype=object)
r=0
for s in range(len(snaps)-nrows,len(snaps)):
    c=0
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        if(v==0): 
            show_ylabels = True
            name = masss[s]
        else: 
            show_ylabels = False
            name = None
        filename = ionfrac.mass_ionfrac_const_pcolormesh(snaps[s], years[s], scale, vmin=VMIN, vmax=VMAX, v=v,
                               show_xlabels=show_xlabels, show_ylabels=show_ylabels,
                               name=name, xx_name=0.02, yy_name=0.05, namefs=30)
        imgfiles[r,c] = filename
        c+=1
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        show_ylabels = False
        filename = ionfrac.vol_ionfrac_const_pcolormesh(snaps[s], years[s], scale, vmin=VMIN, vmax=VMAX, v=v,
                              show_xlabels=show_xlabels, show_ylabels=show_ylabels)
        imgfiles[r,c] = filename
        c+=1
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        show_ylabels = False
        filename = ionfrac.emis_ionfrac_const_pcolormesh(snaps[s], years[s], scale, f=5, vmin=VMIN, vmax=VMAX, v=v,
                               show_xlabels=show_xlabels, show_ylabels=show_ylabels)
        imgfiles[r,c] = filename
        c+=1
    r+=1

imgfiles_arr_4000au_ratio = imgfiles

In [ ]:
allimage = plot.ionfrac_pil_image(imgfiles_arr_4000au_ratio,horiz_cname, extracrop=horiz_crop, debug=False)
display(allimage)
name = VICO_loc+'/figures/ionfrac_maps/ionfrac_caseB_4000au.png' if INCLUDE_4000 else VICO_loc+'/figures/ionfrac_maps/ionfrac_cooling_4000au.png'
allimage.save(name)

#### 4B: 4000 au Cooling with contours

In [ ]:
nrows = len(snaps) if INCLUDE_4000 else len(snaps) - 1
ncols = 9
scale = 4000
show_contours=True
lw_cont = 3
colors_cont = ['#8080ff', '#ff99ff', 'white']


imgfiles = np.empty((nrows, ncols), dtype=object)
r=0
for s in range(len(snaps)-nrows,len(snaps)):
    c=0
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        if(v==0): 
            show_ylabels = True
            name = masss[s]
        else: 
            show_ylabels = False
            name = None
        filename = ionfrac.mass_ionfrac_pcolormesh(snaps[s], years[s], scale, vmin=VMIN, vmax=VMAX, v=v,
                               show_xlabels=show_xlabels, show_ylabels=show_ylabels,
                               name=name, xx_name=0.02, yy_name=0.05, namefs=30, 
                               show_contours=show_contours, lw_cont=lw_cont, colors_cont=colors_cont)
        imgfiles[r,c] = filename
        c+=1
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        show_ylabels = False
        filename = ionfrac.vol_ionfrac_pcolormesh(snaps[s], years[s], scale, vmin=VMIN, vmax=VMAX, v=v,
                              show_xlabels=show_xlabels, show_ylabels=show_ylabels, 
                               show_contours=show_contours, lw_cont=lw_cont, colors_cont=colors_cont)
        imgfiles[r,c] = filename
        c+=1
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        show_ylabels = False
        filename = ionfrac.emis_ionfrac_pcolormesh(snaps[s], years[s], scale, f=5, vmin=VMIN, vmax=VMAX, v=v,
                               show_xlabels=show_xlabels, show_ylabels=show_ylabels, 
                               show_contours=show_contours, lw_cont=lw_cont, colors_cont=colors_cont)
        imgfiles[r,c] = filename
        c+=1
    r+=1

imgfiles_arr_4000au_ratio_contours = imgfiles

In [ ]:
allimage = plot.ionfrac_pil_image(imgfiles_arr_4000au_ratio_contours, horiz_cname, extracrop=horiz_crop, debug=False)
display(allimage)
name = VICO_loc+'/figures/ionfrac_maps/ionfrac_caseB_4000au_contours.png' if INCLUDE_4000 else VICO_loc+'/figures/ionfrac_maps/ionfrac_cooling_4000au_contours.png'
allimage.save(name )